In [43]:
!git clone https://github.com/franciscogarate/utils_ucm

fatal: destination path 'utils_ucm' already exists and is not an empty directory.


In [44]:
!pip install pyliferisk

In [45]:
import pandas as pd
import numpy as np

In [46]:
from utils_ucm import clr

In [47]:
from pyliferisk import MortalityTable, lx
from pyliferisk.mortalitytables import PASEM2020_Decesos_M_2ord
mt = MortalityTable(qx=PASEM2020_Decesos_M_2ord)

In [48]:
edad = 50
df = pd.DataFrame(pd.date_range(start='2024-12-31',periods=(mt.w -edad),freq='YE'), columns=['Fecha'])
df['edad'] = edad + df.index
df['t'] = df.index
df['lx'] = df['edad'].apply(lambda x: mt.lx[x+1] if x <= mt.w else 0)
df.head(5)

,Fecha,edad,t,lx
0,2024-12-31,50,0,97012.403177
1,2025-12-31,51,1,96701.888166
2,2026-12-31,52,2,96359.083186
3,2027-12-31,53,3,95982.230416
4,2028-12-31,54,4,95569.680674


In [49]:
df['qx'] = df['lx'].diff(-1).fillna(0)/df['lx'][0]
df['sum_qx'] = df['qx'].cumsum()
df['px'] = 1 - df['sum_qx']
df.head(5)

,Fecha,edad,t,lx,qx,sum_qx,px
0,2024-12-31,50,0,97012.403177,0.003201,0.003201,0.996799
1,2025-12-31,51,1,96701.888166,0.003534,0.006734,0.993266
2,2026-12-31,52,2,96359.083186,0.003885,0.010619,0.989381
3,2027-12-31,53,3,95982.230416,0.004253,0.014872,0.985128
4,2028-12-31,54,4,95569.680674,0.004638,0.019510,0.980490


In [50]:
def incr_capital(capital, t):
  return capital * (1 + 0.015) ** t

df['capital'] = incr_capital(5000, df.t)
df.head(5)

,Fecha,edad,t,lx,qx,sum_qx,px,capital
0,2024-12-31,50,0,97012.403177,0.003201,0.003201,0.996799,5000.000000
1,2025-12-31,51,1,96701.888166,0.003534,0.006734,0.993266,5075.000000
2,2026-12-31,52,2,96359.083186,0.003885,0.010619,0.989381,5151.125000
3,2027-12-31,53,3,95982.230416,0.004253,0.014872,0.985128,5228.391875
4,2028-12-31,54,4,95569.680674,0.004638,0.019510,0.980490,5306.817753


In [51]:
df['pagos'] = df['capital'] * df['qx']
df.head(5)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos
0,2024-12-31,50,0,97012.403177,0.003201,0.003201,0.996799,5000.000000,16.003882
1,2025-12-31,51,1,96701.888166,0.003534,0.006734,0.993266,5075.000000,17.933122
2,2026-12-31,52,2,96359.083186,0.003885,0.010619,0.989381,5151.125000,20.009975
3,2027-12-31,53,3,95982.230416,0.004253,0.014872,0.985128,5228.391875,22.233979
4,2028-12-31,54,4,95569.680674,0.004638,0.019510,0.980490,5306.817753,24.614583


In [52]:
df['qx'].sum()

0.9999999998027537

In [53]:
df['caida'] = 0.03
df['polizas'] = (1 - df['caida']).cumprod()
df.head()

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas
0,2024-12-31,50,0,97012.403177,0.003201,0.003201,0.996799,5000.000000,16.003882,0.03,0.970000
1,2025-12-31,51,1,96701.888166,0.003534,0.006734,0.993266,5075.000000,17.933122,0.03,0.940900
2,2026-12-31,52,2,96359.083186,0.003885,0.010619,0.989381,5151.125000,20.009975,0.03,0.912673
3,2027-12-31,53,3,95982.230416,0.004253,0.014872,0.985128,5228.391875,22.233979,0.03,0.885293
4,2028-12-31,54,4,95569.680674,0.004638,0.019510,0.980490,5306.817753,24.614583,0.03,0.858734


In [54]:
df['clr'] = df['t'].apply(lambda t: clr[t])
df['factor_desc'] = df.apply(lambda x: 1 / (1 + x.clr) ** (x.t), axis=1)
df['polizas_benef'] =  df['px'] * df['polizas']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas,clr,factor_desc,polizas_benef
0,2024-12-31,50,0,97012.403177,0.003201,0.003201,0.996799,5000.0,16.003882,0.03,0.9700,0.02236,1.000000,0.966895
1,2025-12-31,51,1,96701.888166,0.003534,0.006734,0.993266,5075.0,17.933122,0.03,0.9409,0.02093,0.979499,0.934564


In [55]:
df['pagos_prob'] = df['pagos'] * df['polizas_benef']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas,clr,factor_desc,polizas_benef,pagos_prob
0,2024-12-31,50,0,97012.403177,0.003201,0.003201,0.996799,5000.0,16.003882,0.03,0.9700,0.02236,1.000000,0.966895,15.474077
1,2025-12-31,51,1,96701.888166,0.003534,0.006734,0.993266,5075.0,17.933122,0.03,0.9409,0.02093,0.979499,0.934564,16.759643


In [56]:
df['gastos'] = df['capital'] * 0.01
df['gastos_prob'] = df['gastos'] * df['polizas_benef']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas,clr,factor_desc,polizas_benef,pagos_prob,gastos,gastos_prob
0,2024-12-31,50,0,97012.403177,0.003201,0.003201,0.996799,5000.0,16.003882,0.03,0.9700,0.02236,1.000000,0.966895,15.474077,50.00,48.344762
1,2025-12-31,51,1,96701.888166,0.003534,0.006734,0.993266,5075.0,17.933122,0.03,0.9409,0.02093,0.979499,0.934564,16.759643,50.75,47.429103


In [57]:
df['salidas_prob'] = df['pagos_prob'] + df['gastos_prob']
df.head(2)

,Fecha,edad,t,lx,qx,sum_qx,px,capital,pagos,caida,polizas,clr,factor_desc,polizas_benef,pagos_prob,gastos,gastos_prob,salidas_prob
0,2024-12-31,50,0,97012.403177,0.003201,0.003201,0.996799,5000.0,16.003882,0.03,0.9700,0.02236,1.000000,0.966895,15.474077,50.00,48.344762,63.818840
1,2025-12-31,51,1,96701.888166,0.003534,0.006734,0.993266,5075.0,17.933122,0.03,0.9409,0.02093,0.979499,0.934564,16.759643,50.75,47.429103,64.188746


In [58]:
df.salidas_prob @ df.factor_desc

1828.1312249612374